___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Проверочный проект по линейной регрессии

Мы изучили построение признаков, кросс-валидацию и поиск по сетке. Пришло время проверить Ваши новые навыки. Добро пожаловать на проверочный проект по машинному обучению! В этом проекте у нас будут достаточно подробные пошаговые инструкции. Будущие проекты будут уже в более свободной форме. Мы начнём проект с финальной версии набора данных Ames Housing, с которым мы работали в разделе "построение признаков" этого курса. Цель этого проекта - создать модель линейной регрессии, обучить её на данных с поиском оптимальных значений параметров по сетке, и затем оценить модель на тестовом наборе данных.

---
---
---
## Выполните задания, написанные жирным шрифтом

**ЗАДАНИЕ: Выполните ячейки ниже для импорта библиотек и загрузки данных. Возможно в будущем Вам понадобятся дополнительные команды import от scikit-learn.**

### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Данные

In [3]:
df = pd.read_csv("../DATA/AMES_Final_DF.csv")

In [4]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


**ЗАДАНИЕ: Мы будем пытаться спрогнозировать значение колонки SalePrice. Разделите данные на две части - признаки X и целевая переменная y.**

In [6]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [7]:
X.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [8]:
y.head()

0    215000
1    105000
2    172000
3    244000
4    189900
Name: SalePrice, dtype: int64

**ЗАДАНИЕ: С помощью scikit-learn разделите и X, и y на обучающий и тестовый наборы данных. Поскольку далее мы будем использовать поиск по сетке, то выделите под тестовые данные 10% от всех данных. Чтобы получить такое же разбиение данных, как и в нашем блокноте, можете использовать random_state = 101.**

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

**ЗАДАНИЕ: Признаки в этом наборе данных имеют различный масштаб и различные единицы измерения. Для оптимальной работы регрессии, выполните масштабирование признаков X. Обратите внимание на то, какие данные подавать на вход для .fit(), а какие данные на вход для .transform().**

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#Standard scaler makes data normally distributed
# for example, if we have a feature with a mean of 1000 and a std of 200
# after standard scaling, the mean will be 0 and the std will be 1
# this is important for neural networks because they perform better with normally distributed data
# this is because the weights are initialized randomly and if the data is not normally distributed, the weights will be updated in a way that is not optimal

**ЗАДАНИЕ: Мы будем использовать модель "Elastic Net". Создайте экземпляр модели ElasticNet в scikit-learn, используя параметры по умолчанию.**

In [14]:
from sklearn.linear_model import ElasticNet

In [15]:
elasticnet_model = ElasticNet()

**ЗАДАНИЕ: Модель Elastic Net работает с двумя основными параметрами - alpha и L1_ratio. Создайте словарь с набором различных значений этих параметров, для поиска по сетке. Вы можете выбрать различные значения параметров, но имейте ввиду, что Ваши результаты могут не совпасть с результатами в блокноте с решениями.**

In [16]:
param_grid = {
    'alpha': [.0001, .001, .01, .1, 1, 5, 10, 50, 100],
    'l1_ratio': [0, .1, .3, .5, .7, .9, .99, 1]
}

**ЗАДАНИЕ: С помощью scikit-learn создайте объект GridSearchCV и запустите поиск по сетке для нахождения наилучших параметров модели, используя обучающие данные (предварительно смасштабированные). [Для некоторых комбинаций параметров Вы можете получить предупреждения (warnings).](https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge)**

In [17]:
from sklearn.model_selection import GridSearchCV
grid_model = GridSearchCV(estimator=elasticnet_model, param_grid=param_grid,
                            scoring='neg_mean_squared_error', cv=5, verbose=2)

In [18]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   0.1s


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.561e+11, tolerance: 1.355e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.648e+11, tolerance: 1.308e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in s

[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   0.1s
[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   0.1s


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.136e+11, tolerance: 1.415e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.096e+11, tolerance: 1.438e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in s

[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   0.1s
[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   0.1s


...

d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.414e+12, tolerance: 1.438e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV] END ..............................alpha=100, l1_ratio=0; total time=   0.2s


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.024e+12, tolerance: 1.346e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV] END ..............................alpha=100, l1_ratio=0; total time=   0.2s
[CV] END ............................alpha=100, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.3; total time=   0.0s
[CV] END ............................alpha=100, l1_ratio=0.5; total time=   0.0s
[CV] END ...................

,estimator,ElasticNet()
,param_grid,"{'alpha': [0.0001, 0.001, ...], 'l1_ratio': [0, 0.1, ...]}"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,100


**ЗАДАНИЕ: Отобразите наилучшую комбинацию параметров для Вашей модели.**

In [19]:
grid_model.best_estimator_

,alpha,100
,l1_ratio,1
,fit_intercept,True
,precompute,False
,max_iter,1000
,copy_X,True
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [325]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

In [20]:
print(grid_model.best_estimator_.coef_)


print(F"{(grid_model.best_estimator_.coef_ == 0).sum()} is the number of features eliminated by the model")

[ 7.59577749e+02  4.92687908e+03  1.01952226e+04  6.09731784e+03
  1.15365091e+04  1.97393916e+03  5.24817166e+03  9.49356600e+03
  1.93487084e+03 -0.00000000e+00  8.86132315e+03  0.00000000e+00
  3.33997062e+03 -8.70962616e+02  2.80936145e+04  6.19644761e+02
 -1.43046809e+02  1.32989733e+03  9.36634149e+02 -3.52079447e+03
 -2.14629753e+03  0.00000000e+00  2.50759498e+03 -0.00000000e+00
  2.33417158e+03  3.12445967e+03  1.02725463e+03  2.21167442e+02
  2.27644633e+02 -0.00000000e+00  2.19217686e+03  1.81607337e+03
  1.91134676e+02 -3.46094014e+02 -5.67982880e+02 -4.79249415e+02
 -1.13562739e+03  0.00000000e+00 -0.00000000e+00  1.58002616e+03
  1.70710549e+03  6.77675316e+02  8.25571222e+02  4.35590658e+02
 -0.00000000e+00  1.33175932e+03 -0.00000000e+00 -0.00000000e+00
  2.94188895e+02 -2.21529342e+03 -5.27229533e+02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  6.61417500e+00 -1.92598214e+03
  9.69452184e+02  4.39780082e+02 -0.00000000e+00  3.99472774e+02
  1.10639788e+03 -8.58799

**ЗАДАНИЕ: Оцените работу модели на тестовом наборе данных (предварительно смасштабированном) в 10%, которые модель ещё не видела. В блокноте с решениями мы получили MAE = $\$$14149 и RMSE = $\$$20532**

In [21]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [22]:
predictions = grid_model.predict(X_test)

In [23]:
mean_absolute_error(y_test, predictions)

14195.35490056217

In [24]:
root_mean_squared_error(y_test, predictions)

20558.50856689317

In [25]:
np.mean(df['SalePrice'])

np.float64(180815.53743589742)

14149.055026374837

20532.890234901013

180815.53743589742

## Отличная работа!

----

In [95]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

In [96]:
X_train_to_poly = X_train.iloc[:,:37]
X_train_dummies = X_train.iloc[:, 37:]
X_test_to_poly = X_test.iloc[:, :37]
X_test_dummies = X_test.iloc[:, 37:]

# Separating the dummies to avoid polynomial expansion on them

In [114]:
from sklearn.preprocessing import PolynomialFeatures

poly_converter = PolynomialFeatures(degree=3, include_bias=False)

poly_converter.fit(X_train_to_poly)
X_train_poly_no_dummies = poly_converter.transform(X_train_to_poly)
X_test_poly_no_dummies = poly_converter.transform(X_test_to_poly)

#Converting to polynomial features of degree 2 to capture non-linear relationships

In [115]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_poly_no_dummies)
X_train_poly_no_dummies = scaler.transform(X_train_poly_no_dummies)
X_test_poly_no_dummies = scaler.transform(X_test_poly_no_dummies)

#Scaling the data again after polynomial feature expansion
# this is important because the polynomial features are not normally distributed
# they can have a very high range of values
# this can cause the model to perform poorly
# if we don't scale the data again, the model will be biased towards the features with a higher range of values

#Standard scaler makes data normally distributed
# for example, if we have a feature with a mean of 1000 and a std of 200
# after standard scaling, the mean will be 0 and the std will be 1


In [116]:
X_train_poly_no_dummies

array([[-0.8384808 , -0.50803335, -0.05639001, ...,  0.        ,
         0.        , -0.21472058],
       [-1.03015138, -0.50803335,  1.35711957, ...,  0.        ,
         0.        , -0.21472058],
       [ 1.26989557,  0.06576361,  0.65036478, ...,  0.        ,
         0.        , -0.21472058],
       ...,
       [-0.45513964, -0.42320424, -0.05639001, ...,  0.        ,
         0.        , -0.21472058],
       [ 0.07195445, -0.42000314, -0.7631448 , ...,  0.        ,
         0.        , -0.21472058],
       [ 0.0880722 ,  0.30477899,  1.35711957, ...,  0.        ,
         0.        , -0.21472058]], shape=(2632, 9879))

In [101]:
X_train_poly_no_dummies

array([[-0.8384808 , -0.50803335, -0.05639001, ..., -0.01949573,
         0.        , -0.21472058],
       [-1.03015138, -0.50803335,  1.35711957, ..., -0.01949573,
         0.        , -0.21472058],
       [ 1.26989557,  0.06576361,  0.65036478, ..., -0.01949573,
         0.        , -0.21472058],
       ...,
       [-0.45513964, -0.42320424, -0.05639001, ..., -0.01949573,
         0.        , -0.21472058],
       [ 0.07195445, -0.42000314, -0.7631448 , ..., -0.01949573,
         0.        , -0.21472058],
       [ 0.0880722 ,  0.30477899,  1.35711957, ..., -0.01949573,
         0.        , -0.21472058]], shape=(2632, 740))

In [117]:
X_train_poly = pd.concat([pd.DataFrame(X_train_poly_no_dummies, index=X_train_to_poly.index, columns=poly_converter.get_feature_names_out()), X_train_dummies], axis=1)
X_test_poly = pd.concat([pd.DataFrame(X_test_poly_no_dummies, index=X_test_to_poly.index, columns=poly_converter.get_feature_names_out()), X_test_dummies], axis=1)

#Concatenating the polynomial features with the dummies dataframes to create the final training and testing dataframes for the model 

#asigning the index of X_train_dummies to X_train_poly_no_dummies to make sure that the index is the same
# this is important because we will be concatenating the two dataframes later. If the index is not the same, the concatenation will not work as expected. 
# For example, if the index of X_train_dummies is [0, 1, 2] and the index of X_train_poly_no_dummies is [3, 4, 5],
# the concatenation will result in a dataframe with an index of [0, 1, 2, 3, 4, 5] instead of [0, 1, 2].

In [375]:
#X_train_dummies.index

In [376]:
#poly_converter.get_feature_names_out()

In [118]:
from sklearn.linear_model import ElasticNet
elasticnet_model = ElasticNet()

#Using the same model and hyperparameters as before to see if polynomial features improve the model performance

In [125]:
X_train_poly

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
2670,-0.838481,-0.508033,-0.056390,0.387114,-1.232140,-1.629914,-0.561426,-0.993920,-0.296191,0.592342,...,0,0,0,0,1,0,0,0,1,0
1075,-1.030151,-0.508033,1.357120,-0.505079,1.155828,1.097803,0.419343,0.929472,-0.296191,-0.137859,...,0,0,0,0,1,0,0,0,1,0
1106,1.269896,0.065764,0.650365,-0.505079,0.724667,0.475692,3.066305,1.672911,-0.296191,-0.954772,...,0,0,0,0,1,0,0,0,1,0
1089,0.225636,-0.271952,0.650365,-0.505079,1.023163,0.906384,-0.561426,-0.993920,-0.296191,0.647107,...,0,0,0,0,1,0,0,0,1,0
569,-0.237144,-0.777859,-0.056390,-0.505079,1.222160,1.145658,0.842857,1.193199,-0.296191,-0.852088,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,-0.455140,-0.348512,-1.469900,1.279307,-0.369819,-1.151367,-0.561426,0.674839,-0.296191,-0.904571,...,0,0,0,0,1,0,0,0,1,0
1599,1.461566,1.560677,-0.763145,-0.505079,-0.568816,-1.438495,-0.561426,-0.993920,-0.296191,0.984825,...,0,0,0,0,1,0,0,0,1,0
1361,-0.455140,-0.423204,-0.056390,0.387114,-1.099476,-1.247076,-0.561426,-0.993920,-0.296191,1.632879,...,0,0,0,0,0,0,0,0,1,0
1547,0.071954,-0.420003,-0.763145,0.387114,-1.033143,-1.390640,-0.561426,-0.077694,0.440407,-0.785913,...,0,0,0,0,1,0,0,0,1,0


In [121]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [.0001, .001, .01, .1, 1, 5, 10, 50, 100],
    'l1_ratio': [0, .1, .3, .5, .7, .9, .99, 1]
}

grid_model = GridSearchCV(estimator=elasticnet_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=2)

# Using GridSearchCV to find the best hyperparameters for the model
# this is important because the hyperparameters can have a big impact on the model performance


In [122]:
grid_model.fit(X_train_poly, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.208e+10, tolerance: 1.355e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   9.9s


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.847e+10, tolerance: 1.308e+09
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV] END ...........................alpha=0.0001, l1_ratio=0; total time=   9.9s


...

[CV] END ..............................alpha=100, l1_ratio=1; total time=   6.1s


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.220e+10, tolerance: 1.346e+09
  model = cd_fast.enet_coordinate_descent(


[CV] END ..............................alpha=100, l1_ratio=1; total time=   6.0s


d:\PYTHON DATA SCIENCE\Stepik Course\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.575e+10, tolerance: 1.716e+09
  model = cd_fast.enet_coordinate_descent(


,estimator,ElasticNet()
,param_grid,"{'alpha': [0.0001, 0.001, ...], 'l1_ratio': [0, 0.1, ...]}"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,100


In [124]:
mean_absolute_error(y_test, grid_model.predict(X_test_poly)), root_mean_squared_error(y_test, grid_model.predict(X_test_poly))

(14599.582388117331, 21488.18105396896)

In [353]:

grid_model

,estimator,ElasticNet()
,param_grid,"{'alpha': [0.0001, 0.001, ...], 'l1_ratio': [0, 0.1, ...]}"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,100


In [81]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

param_dist = {
    'alpha': scipy.stats.uniform(0.0001, 10), 
    'l1_ratio': scipy.stats.uniform(0, 1)
}

r_grid_model = RandomizedSearchCV(estimator=elasticnet_model, param_distributions=param_dist, cv=5, scoring='neg_mean_squared_error', verbose=2)

In [82]:
r_grid_model.fit(X_train_poly, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END alpha=1.55856257008323, l1_ratio=0.32636812758470035; total time=   0.0s
[CV] END alpha=1.55856257008323, l1_ratio=0.32636812758470035; total time=   0.0s
[CV] END alpha=1.55856257008323, l1_ratio=0.32636812758470035; total time=   0.0s
[CV] END alpha=1.55856257008323, l1_ratio=0.32636812758470035; total time=   0.0s
[CV] END alpha=1.55856257008323, l1_ratio=0.32636812758470035; total time=   0.0s
[CV] END alpha=7.790800039623573, l1_ratio=0.3791506918755755; total time=   0.0s
[CV] END alpha=7.790800039623573, l1_ratio=0.3791506918755755; total time=   0.0s
[CV] END alpha=7.790800039623573, l1_ratio=0.3791506918755755; total time=   0.0s
[CV] END alpha=7.790800039623573, l1_ratio=0.3791506918755755; total time=   0.0s
[CV] END alpha=7.790800039623573, l1_ratio=0.3791506918755755; total time=   0.0s
[CV] END alpha=8.172398797068976, l1_ratio=0.40717362096050214; total time=   0.0s
[CV] END alpha=8.172398797068976, l1

,estimator,ElasticNet()
,param_distributions,"{'alpha': <scipy.stats....002198A917380>, 'l1_ratio': <scipy.stats....002198AAF6990>}"
,n_iter,10
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [83]:
r_grid_model.best_params_

{'alpha': np.float64(0.34553101395466124),
 'l1_ratio': np.float64(0.04937299696621744)}

In [85]:
mean_absolute_error(y_test, r_grid_model.predict(X_test_poly))

16941.927087803397

In [86]:
root_mean_squared_error(y_test, r_grid_model.predict(X_test_poly))

23772.945939266967